In [ ]:
%%capture
!pip install unsloth langchain langchain-huggingface faiss-cpu langchain-community

In [ ]:
import shutil
# I unzip the zip file containing the model
shutil.unpack_archive("lora_pharma.zip", "lora_pharma", "zip")

In [ ]:
from unsloth import FastLanguageModel
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
# I load the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_pharma",
    max_seq_length = 2048,
    load_in_4bit = True,
)

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
# I load the vector database (FAISS)
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

vector_db = FAISS.load_local("faiss_pharmacy", embeddings, allow_dangerous_deserialization=True)

In [ ]:
def pharmaceutical_question(question, k=5, critical_threshold=1.5):

    # Retrieval
    docs_with_score = vector_db.similarity_search_with_score(question, k=k)

    # I extract the best score (the one of the first document)
    best_score = docs_with_score[0][1]

    # Validation of the threshold
    if best_score > critical_threshold:
        return "The question is out of my domain, and I cannot find any related information.", []

    # If it passes the filter, I prepare the context
    docs = [doc for doc, score in docs_with_score]
    retrieved_context = "\n\n".join([doc.page_content for doc in docs])

    # Prompt Preparation
    messages = [
    {"role": "system", "content": "You are a precise pharmaceutical assistant. If the answer is not in the context, say that you do not know, do not invent."},
    {"role": "user", "content": f"Context: {retrieved_context}\n\nQuestion: {question}"},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text=prompt, return_tensors="pt").to("cuda")

    #  Generation
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        use_cache=True,
        temperature=0.1
    )

    answer = tokenizer.batch_decode(outputs)[0].split("<start_of_turn>model\n")[-1]
    return answer.replace("<end_of_turn>", "").replace("<eos>", "").strip(), docs

print("Pipeline configured")

Pipeline configured


In [ ]:
user_question = "What is the composition of Aciloc 150?"

answer, sources = pharmaceutical_question(user_question)

print(f"\n--- QUESTION ---\n{user_question}")
print(f"\n--- ASSISTANT'S RESPONSE ---\n{answer}")
print("\n--- SOURCES ---")
for i, doc in enumerate(sources):
    print(f"Fuente {i+1}: {doc.metadata.get('source', 'Unknown')}")


--- QUESTION ---
What is the composition of Aciloc 150?

--- ASSISTANT'S RESPONSE ---
The composition of Aciloc 150 is Ranitidine (150mg).

--- SOURCES ---
Fuente 1: Kaggle_Dataset
Fuente 2: Kaggle_Dataset
Fuente 3: Kaggle_Dataset
Fuente 4: Kaggle_Dataset
Fuente 5: Kaggle_Dataset


In [ ]:
user_question = "What are the contraindications of Azithromycin?"

answer, sources = pharmaceutical_question(user_question)

print(f"\n--- QUESTION ---\n{user_question}")
print(f"\n--- ASSISTANT'S RESPONSE ---\n{answer}")
print("\n--- SOURCES ---")
for i, doc in enumerate(sources):
    print(f"Fuente {i+1}: {doc.metadata.get('source', 'Unknown')}")


--- QUESTION ---
What are the contraindications of Azithromycin?

--- ASSISTANT'S RESPONSE ---
The provided context does not contain information about the contraindications of Azithromycin. It only mentions the uses and side effects of Azithromycin in the context of Azithral 200 Liquid and ATM 200 Oral Suspension, which are both formulations of Azithromycin. Therefore, I cannot answer the question about the contraindications of Azithromycin based on the given context.

--- SOURCES ---
Fuente 1: Katzung - Basic and Clinical Pharmacology 12th Edition (2012).pdf
Fuente 2: Kaggle_Dataset
Fuente 3: Kaggle_Dataset
Fuente 4: Kaggle_Dataset
Fuente 5: Kaggle_Dataset


In [ ]:
user_question = "Who is Messi?"

answer, sources = pharmaceutical_question(user_question)

print(f"\n--- QUESTION ---\n{user_question}")
print(f"\n--- ASSISTANT'S RESPONSE ---\n{answer}")
print("\n--- SOURCES ---")
for i, doc in enumerate(sources):
    print(f"Fuente {i+1}: {doc.metadata.get('source', 'Unknown')}")


--- QUESTION ---
Who is Messi?

--- ASSISTANT'S RESPONSE ---
The question is out of my domain, and I cannot find any related information.

--- SOURCES ---
